In [7]:
import pandas as pd
import os,h5py
import pandas as pd
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
from collections import OrderedDict
from sklearn.metrics import r2_score
%matplotlib inline  

os.chdir('/data/arpit.goel/20_TwoSigmaFinancial')

In [2]:
df = pd.read_hdf('01.RawData/train.h5')

In [24]:
cols = ['technical_20']

train=df[df['timestamp']<1000][cols+['y','timestamp']]
test=df[df['timestamp']>=1000][cols+['y','timestamp']]

low_y_cut = -0.086093
high_y_cut = 0.093497

y_values_within = ((train['y'] > low_y_cut) & (train['y'] <high_y_cut))
train_cut = train.loc[y_values_within,:]

mean_vals = train_cut.median()
train_cut.fillna(mean_vals,inplace=True)
train.fillna(mean_vals,inplace=True)
test.fillna(mean_vals,inplace=True)

from sklearn.linear_model import LinearRegression
model1 = LinearRegression()
model1.fit(train_cut[cols],train_cut['y'])

train['prediction']=model1.predict(train[cols]).clip(low_y_cut,high_y_cut)
test['prediction']=model1.predict(test[cols]).clip(low_y_cut,high_y_cut)

In [27]:
predictions=pd.concat([train,test])
predictions['prediction1']=predictions['prediction']-0.001
predictions['prediction2']=predictions['prediction']
predictions['prediction3']=predictions['prediction']+0.001

def get_daily_summary(group):
    output=pd.Series({})
    output['y']=group['y'].mean()
    output['avg_prediction1']=group['prediction1'].mean()
    output['avg_prediction2']=group['prediction2'].mean()
    output['avg_prediction3']=group['prediction3'].mean()
    output['r21']=r2_score(group['y'], group['prediction1'])
    output['r22']=r2_score(group['y'], group['prediction2'])
    output['r23']=r2_score(group['y'], group['prediction3'])
    return pd.Series(output)

predictions.groupby('timestamp').apply(get_daily_summary).to_csv('02.Profiling/07.MarketMean.csv')


In [16]:
df1=df[['y','timestamp']]
df1['avg_prediction']=df['y'].mean()

def get_daily_summary(group):
    output=pd.Series({})
    output['y']=group['y'].mean()
    output['avg_prediction1']=group['avg_prediction'].mean()*0.9
    output['avg_prediction2']=group['avg_prediction'].mean()
    output['avg_prediction3']=group['avg_prediction'].mean()*1.1
    output['r21']=r2_score(group['y'], group['avg_prediction']*0.9)
    output['r22']=r2_score(group['y'], group['avg_prediction'])
    output['r23']=r2_score(group['y'], group['avg_prediction']*1.1)
    return pd.Series(output)

df1.groupby('timestamp').apply(get_daily_summary).to_csv('02.Profiling/07.MarketMean.csv')
!head 02.Profiling/07.MarketMean.csv

/data/arpit.goel/anacondainstall/lib/python2.7/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


timestamp,y,avg_prediction1,avg_prediction2,avg_prediction3,r21,r22,r23
0,-0.00186999782454,0.000199575786246,0.000221750873607,0.000243925960967,-0.00604053380999,-0.00617067369963,-0.00630220058281
1,0.000500668713357,0.000199575786246,0.000221750873607,0.000243925960967,-0.000112371993624,-9.64285328175e-05,-8.17041791372e-05
2,-0.00578675512224,0.000199575786246,0.000221750873607,0.000243925960967,-0.0418532242068,-0.0421638712406,-0.042475666873
3,0.00163280090783,0.000199575786246,0.000221750873607,0.000243925960967,-0.00320501894038,-0.00310660904488,-0.00300973363463
4,-0.00430873036385,0.000199575786246,0.000221750873607,0.000243925960967,-0.0157657614917,-0.0159212374502,-0.0160774762748
5,0.00374813936651,0.000199575786246,0.000221750873607,0.000243925960967,-0.0185965987828,-0.0183649036191,-0.0181346608639
6,-0.00665966747329,0.000199575786246,0.000221750873607,0.000243925960967,-0.0508461630898,-0.0511754530601,-0.051505805866
7,-0.00356772681698,0.000199575786246,0.00022

In [36]:
all_timestamps= df['timestamp'].drop_duplicates().sort_values().tolist()
train_timestamp_cutoff=all_timestamps[int(0.6*len(all_timestamps))]
print  train_timestamp_cutoff

1087
